In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
data = pd.read_csv('Aggregated User Data.csv')
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

In [3]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [4]:
data.iloc[:,1:27] = interpolation(data.iloc[:,1:27])
data = data.iloc[:,:27].values
data

array([['00EABED2-271D-49D8-B599-1D4A09240601', 0.996815, 0.003529, ...,
        0.10692, 0.516842, 0.255494],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996864, 0.004172, ...,
        -0.079483, 0.357748, 0.036252],
       ['00EABED2-271D-49D8-B599-1D4A09240601', 0.996825, 0.003667, ...,
        0.122432, 0.464881, 0.222375],
       ...,
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.012776, 0.008135, ...,
        -0.099042, -0.307551, -0.592356],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.013261, 0.013788, ...,
        0.03825, -0.251999, -0.714921],
       ['0BFC35E2-4817-4865-BFA7-764742302A2D', 1.004676,
        0.016819999999999998, ..., 0.566422, -0.23814, -0.670271]],
      dtype=object)

In [5]:
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
data = np.array(ct.fit_transform(data))
data

array([[1.0, 0.0, 0.0, ..., 0.10692, 0.516842, 0.255494],
       [1.0, 0.0, 0.0, ..., -0.079483, 0.357748, 0.036252],
       [1.0, 0.0, 0.0, ..., 0.122432, 0.464881, 0.222375],
       ...,
       [0.0, 0.0, 1.0, ..., -0.099042, -0.307551, -0.592356],
       [0.0, 0.0, 1.0, ..., 0.03825, -0.251999, -0.714921],
       [0.0, 0.0, 1.0, ..., 0.566422, -0.23814, -0.670271]], dtype=object)

In [6]:
X = data[:,3:]
y = data[:,:3]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.39569646, -0.38083705, -0.45635096, ..., -1.10927431,
         0.09968151,  0.59721278],
       [ 0.15119226, -0.32543256, -0.4464613 , ..., -0.53282669,
         0.47223648, -0.21141259],
       [ 0.64872995, -0.10609382, -0.07211521, ..., -1.56628452,
         2.76556612, -1.82379313],
       ...,
       [-0.4336159 , -0.37802321, -0.32148446, ..., -0.40387555,
         0.49350533, -0.27643951],
       [-0.56447513,  0.25114179,  0.49775032, ...,  2.60579987,
        -3.07231733, -2.4134348 ],
       [ 0.30674249, -0.39168908, -0.38611883, ..., -0.36097157,
        -0.04594345,  0.19126533]])

In [8]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [9]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 50),
            classifier_block(50, 100),
            classifier_block(100, 75),
            nn.Linear(75, 3) 
        )
    def forward(self, x):
        return self.network(x)

In [10]:
model = Classifier()
lr = 0.01
n_epochs = 2000
batch_size = 1000

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [11]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        _, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        loss = criterion(preds, encoded_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.001584529876709, Final Batch Loss: 0.6571447849273682
Epoch 2, Loss: 4.5493499636650085, Final Batch Loss: 0.5172848105430603
Epoch 3, Loss: 3.6151908338069916, Final Batch Loss: 0.42901143431663513
Epoch 4, Loss: 2.9689984917640686, Final Batch Loss: 0.35522985458374023
Epoch 5, Loss: 2.4121692776679993, Final Batch Loss: 0.3057062029838562
Epoch 6, Loss: 2.0170060098171234, Final Batch Loss: 0.22648508846759796
Epoch 7, Loss: 1.9119723290205002, Final Batch Loss: 0.24632790684700012
Epoch 8, Loss: 1.7812132686376572, Final Batch Loss: 0.26883047819137573
Epoch 9, Loss: 1.6512185782194138, Final Batch Loss: 0.16439323127269745
Epoch 10, Loss: 1.543417900800705, Final Batch Loss: 0.21459434926509857
Epoch 11, Loss: 1.5247046798467636, Final Batch Loss: 0.21192780137062073
Epoch 12, Loss: 1.4012557715177536, Final Batch Loss: 0.1982126086950302
Epoch 13, Loss: 1.2860131412744522, Final Batch Loss: 0.16299065947532654
Epoch 14, Loss: 1.203938290476799, Final Batch Loss: 

Epoch 112, Loss: 0.45073552429676056, Final Batch Loss: 0.05001884326338768
Epoch 113, Loss: 0.48510563746094704, Final Batch Loss: 0.05916827544569969
Epoch 114, Loss: 0.54730324447155, Final Batch Loss: 0.07612529397010803
Epoch 115, Loss: 0.4953868128359318, Final Batch Loss: 0.03750772029161453
Epoch 116, Loss: 0.5060144886374474, Final Batch Loss: 0.07182075828313828
Epoch 117, Loss: 0.5164320766925812, Final Batch Loss: 0.059798095375299454
Epoch 118, Loss: 0.5570154078304768, Final Batch Loss: 0.07905440777540207
Epoch 119, Loss: 0.5812803916633129, Final Batch Loss: 0.10195975005626678
Epoch 120, Loss: 0.5470082946121693, Final Batch Loss: 0.04616429656744003
Epoch 121, Loss: 0.4654402397572994, Final Batch Loss: 0.06880462914705276
Epoch 122, Loss: 0.538799837231636, Final Batch Loss: 0.1009378433227539
Epoch 123, Loss: 0.5095230527222157, Final Batch Loss: 0.07876843959093094
Epoch 124, Loss: 0.5184548608958721, Final Batch Loss: 0.06405826658010483
Epoch 125, Loss: 0.4853322

Epoch 221, Loss: 0.3821711242198944, Final Batch Loss: 0.06822125613689423
Epoch 222, Loss: 0.34374248050153255, Final Batch Loss: 0.03855051100254059
Epoch 223, Loss: 0.34699673019349575, Final Batch Loss: 0.037575580179691315
Epoch 224, Loss: 0.34258054941892624, Final Batch Loss: 0.03872290626168251
Epoch 225, Loss: 0.342574056237936, Final Batch Loss: 0.019848328083753586
Epoch 226, Loss: 0.35448164120316505, Final Batch Loss: 0.04607740417122841
Epoch 227, Loss: 0.36130987480282784, Final Batch Loss: 0.0415942408144474
Epoch 228, Loss: 0.34440910816192627, Final Batch Loss: 0.04767439886927605
Epoch 229, Loss: 0.4374542124569416, Final Batch Loss: 0.08002610504627228
Epoch 230, Loss: 0.37733281403779984, Final Batch Loss: 0.047946225851774216
Epoch 231, Loss: 0.41808371245861053, Final Batch Loss: 0.041123565286397934
Epoch 232, Loss: 0.4123268947005272, Final Batch Loss: 0.050188567489385605
Epoch 233, Loss: 0.36599452793598175, Final Batch Loss: 0.036418937146663666
Epoch 234, L

Epoch 331, Loss: 0.38166596181690693, Final Batch Loss: 0.0787629708647728
Epoch 332, Loss: 0.34060898050665855, Final Batch Loss: 0.026339685544371605
Epoch 333, Loss: 0.35590788535773754, Final Batch Loss: 0.0508037805557251
Epoch 334, Loss: 0.2749630305916071, Final Batch Loss: 0.03612305968999863
Epoch 335, Loss: 0.2988396678119898, Final Batch Loss: 0.04622383043169975
Epoch 336, Loss: 0.3735030274838209, Final Batch Loss: 0.04097667336463928
Epoch 337, Loss: 0.3494703620672226, Final Batch Loss: 0.02785719931125641
Epoch 338, Loss: 0.30897372402250767, Final Batch Loss: 0.026976080611348152
Epoch 339, Loss: 0.32402194291353226, Final Batch Loss: 0.056061334908008575
Epoch 340, Loss: 0.2767828740179539, Final Batch Loss: 0.028794554993510246
Epoch 341, Loss: 0.3449058812111616, Final Batch Loss: 0.06341144442558289
Epoch 342, Loss: 0.3001311346888542, Final Batch Loss: 0.043018583208322525
Epoch 343, Loss: 0.37014052271842957, Final Batch Loss: 0.07766991853713989
Epoch 344, Loss:

Epoch 439, Loss: 0.31468260660767555, Final Batch Loss: 0.024922754615545273
Epoch 440, Loss: 0.3172251731157303, Final Batch Loss: 0.030488930642604828
Epoch 441, Loss: 0.350909860804677, Final Batch Loss: 0.0801168754696846
Epoch 442, Loss: 0.3081622961908579, Final Batch Loss: 0.03667355328798294
Epoch 443, Loss: 0.3205515220761299, Final Batch Loss: 0.036639515310525894
Epoch 444, Loss: 0.32676177844405174, Final Batch Loss: 0.04725617170333862
Epoch 445, Loss: 0.31096855737268925, Final Batch Loss: 0.04797736555337906
Epoch 446, Loss: 0.2623074874281883, Final Batch Loss: 0.03607453033328056
Epoch 447, Loss: 0.3263595402240753, Final Batch Loss: 0.04028308391571045
Epoch 448, Loss: 0.35007411055266857, Final Batch Loss: 0.06217700615525246
Epoch 449, Loss: 0.2734008152037859, Final Batch Loss: 0.03707280382514
Epoch 450, Loss: 0.3206094466149807, Final Batch Loss: 0.03982662782073021
Epoch 451, Loss: 0.300138121470809, Final Batch Loss: 0.02972639538347721
Epoch 452, Loss: 0.29980

...

Epoch 1520, Loss: 0.3256535604596138, Final Batch Loss: 0.02499222196638584
Epoch 1521, Loss: 0.24109436012804508, Final Batch Loss: 0.025933442637324333
Epoch 1522, Loss: 0.2640150021761656, Final Batch Loss: 0.005009785294532776
Epoch 1523, Loss: 0.3394189979881048, Final Batch Loss: 0.07954764366149902
Epoch 1524, Loss: 0.2823002375662327, Final Batch Loss: 0.041976448148489
Epoch 1525, Loss: 0.3174226526170969, Final Batch Loss: 0.03550126403570175
Epoch 1526, Loss: 0.2824806645512581, Final Batch Loss: 0.03335535526275635
Epoch 1527, Loss: 0.23925400339066982, Final Batch Loss: 0.01912000961601734
Epoch 1528, Loss: 0.28274229913949966, Final Batch Loss: 0.03174959868192673
Epoch 1529, Loss: 0.2513585574924946, Final Batch Loss: 0.06317903846502304
Epoch 1530, Loss: 0.2688646875321865, Final Batch Loss: 0.03569236770272255
Epoch 1531, Loss: 0.2428865022957325, Final Batch Loss: 0.04015233740210533
Epoch 1532, Loss: 0.21999118430539966, Final Batch Loss: 0.007628196384757757
Epoch 1

Epoch 1627, Loss: 0.2091697808355093, Final Batch Loss: 0.025276847183704376
Epoch 1628, Loss: 0.15444305166602135, Final Batch Loss: 0.011095987632870674
Epoch 1629, Loss: 0.20887477789074183, Final Batch Loss: 0.03876424953341484
Epoch 1630, Loss: 0.20035391114652157, Final Batch Loss: 0.019342586398124695
Epoch 1631, Loss: 0.17338252905756235, Final Batch Loss: 0.022367598488926888
Epoch 1632, Loss: 0.21499459352344275, Final Batch Loss: 0.017142580822110176
Epoch 1633, Loss: 0.25642313063144684, Final Batch Loss: 0.04827967286109924
Epoch 1634, Loss: 0.19165729265660048, Final Batch Loss: 0.010321157053112984
Epoch 1635, Loss: 0.22559394501149654, Final Batch Loss: 0.034404296427965164
Epoch 1636, Loss: 0.24641776457428932, Final Batch Loss: 0.028182806447148323
Epoch 1637, Loss: 0.14921482279896736, Final Batch Loss: 0.0169863011687994
Epoch 1638, Loss: 0.18284583929926157, Final Batch Loss: 0.005800730548799038
Epoch 1639, Loss: 0.19782872218638659, Final Batch Loss: 0.0074144499

Epoch 1735, Loss: 0.17902808729559183, Final Batch Loss: 0.01475419383496046
Epoch 1736, Loss: 0.798130925744772, Final Batch Loss: 0.025135956704616547
Epoch 1737, Loss: 0.501910176128149, Final Batch Loss: 0.06476748734712601
Epoch 1738, Loss: 0.49967580288648605, Final Batch Loss: 0.06452534347772598
Epoch 1739, Loss: 0.29327988252043724, Final Batch Loss: 0.0362282358109951
Epoch 1740, Loss: 0.31767173297703266, Final Batch Loss: 0.03733003884553909
Epoch 1741, Loss: 0.29588765650987625, Final Batch Loss: 0.03932740539312363
Epoch 1742, Loss: 0.3510136157274246, Final Batch Loss: 0.02331400103867054
Epoch 1743, Loss: 0.3066059462726116, Final Batch Loss: 0.027035528793931007
Epoch 1744, Loss: 0.3213401697576046, Final Batch Loss: 0.03042376972734928
Epoch 1745, Loss: 0.26661876775324345, Final Batch Loss: 0.048395849764347076
Epoch 1746, Loss: 0.2806365033611655, Final Batch Loss: 0.02504202537238598
Epoch 1747, Loss: 0.29769695177674294, Final Batch Loss: 0.0487433560192585
Epoch 

Epoch 1843, Loss: 0.186633319593966, Final Batch Loss: 0.013691832311451435
Epoch 1844, Loss: 0.22404847759753466, Final Batch Loss: 0.012867956422269344
Epoch 1845, Loss: 0.18609813414514065, Final Batch Loss: 0.010194987989962101
Epoch 1846, Loss: 0.19568057358264923, Final Batch Loss: 0.02111460641026497
Epoch 1847, Loss: 0.21257477812469006, Final Batch Loss: 0.05380447953939438
Epoch 1848, Loss: 0.21556347608566284, Final Batch Loss: 0.021991603076457977
Epoch 1849, Loss: 0.20067905448377132, Final Batch Loss: 0.0412689633667469
Epoch 1850, Loss: 0.20417415723204613, Final Batch Loss: 0.01915179193019867
Epoch 1851, Loss: 0.18310823291540146, Final Batch Loss: 0.014731290750205517
Epoch 1852, Loss: 0.17346097063273191, Final Batch Loss: 0.021105654537677765
Epoch 1853, Loss: 0.21188020892441273, Final Batch Loss: 0.034247685223817825
Epoch 1854, Loss: 0.2084463844075799, Final Batch Loss: 0.029699889943003654
Epoch 1855, Loss: 0.18403965141624212, Final Batch Loss: 0.0253435801714

Epoch 1950, Loss: 0.26698200963437557, Final Batch Loss: 0.02242470718920231
Epoch 1951, Loss: 0.23351958207786083, Final Batch Loss: 0.017115365713834763
Epoch 1952, Loss: 0.24345358181744814, Final Batch Loss: 0.02814871445298195
Epoch 1953, Loss: 0.23377608321607113, Final Batch Loss: 0.04599691554903984
Epoch 1954, Loss: 0.23996866680681705, Final Batch Loss: 0.029116062447428703
Epoch 1955, Loss: 0.22838058695197105, Final Batch Loss: 0.02892058528959751
Epoch 1956, Loss: 0.23406836856156588, Final Batch Loss: 0.023964837193489075
Epoch 1957, Loss: 0.23523972928524017, Final Batch Loss: 0.04089081659913063
Epoch 1958, Loss: 0.19899457972496748, Final Batch Loss: 0.023916123434901237
Epoch 1959, Loss: 0.2329451236873865, Final Batch Loss: 0.019413890317082405
Epoch 1960, Loss: 0.19289260730147362, Final Batch Loss: 0.0212219450622797
Epoch 1961, Loss: 0.2016804488375783, Final Batch Loss: 0.00920837465673685
Epoch 1962, Loss: 0.1752127083018422, Final Batch Loss: 0.0224019661545753

In [12]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    _, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(answers, preds))
    print(metrics.classification_report(answers, preds, digits = 3))

[[416  36   2]
 [ 21 788   5]
 [  5   2 596]]
              precision    recall  f1-score   support

           0      0.941     0.916     0.929       454
           1      0.954     0.968     0.961       814
           2      0.988     0.988     0.988       603

    accuracy                          0.962      1871
   macro avg      0.961     0.958     0.959      1871
weighted avg      0.962     0.962     0.962      1871

